In [5]:
import cv2
import kagglehub
import numpy as np
import pandas as pd
import tkinter as tk
import mediapipe as mp

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle



In [6]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    static_image_mode=False, # Set to True for static images, False for video
    max_num_hands=2, # Maximum number of hands to detect
    min_detection_confidence=0.5, # Minimum detection confidence for hand detection
    min_tracking_confidence=0.5 # Minimum tracking confidence for hand landmarks
)

In [7]:
# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, # Thickness of the lines
                                     circle_radius=1, # Radius of the circles
                                     )

In [8]:

# Khởi tạo webcam
cap = cv2.VideoCapture(0)

# Cử chỉ tay mà bạn muốn thu thập dữ liệu
gestures = ["up", "down", "none"]
data = []

print("Bắt đầu thu thập dữ liệu cử chỉ tay...")

for gesture in gestures:
    print(f"\nHãy thực hiện cử chỉ: {gesture}")
    input("Nhấn Enter để bắt đầu thu thập dữ liệu...")

    num_samples = 0
    while num_samples < 50:  # Thu thập 50 mẫu cho mỗi cử chỉ
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Lấy tọa độ landmark và chuẩn hóa
                landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()

                # Lưu landmark và nhãn
                data.append(np.append(landmarks, gesture))
                num_samples += 1

                # Vẽ landmark lên khung hình để theo dõi
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=hand_landmarks,
                    connections=mp_hands.HAND_CONNECTIONS,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec
                )

        cv2.putText(frame, f"Gesture: {gesture} | Samples: {num_samples}/50", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Data Collection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Lưu dữ liệu vào file CSV
df = pd.DataFrame(data)
df.to_csv("hand_landmarks.csv", index=False)
print("\nThu thập dữ liệu hoàn tất và đã lưu vào hand_landmarks.csv")

cap.release()
cv2.destroyAllWindows()

Bắt đầu thu thập dữ liệu cử chỉ tay...

Hãy thực hiện cử chỉ: up

Hãy thực hiện cử chỉ: down

Hãy thực hiện cử chỉ: down

Hãy thực hiện cử chỉ: none

Hãy thực hiện cử chỉ: none

Thu thập dữ liệu hoàn tất và đã lưu vào hand_landmarks.csv

Thu thập dữ liệu hoàn tất và đã lưu vào hand_landmarks.csv


In [9]:
# Đọc dữ liệu từ file CSV
df = pd.read_csv("hand_landmarks.csv")

# Tách dữ liệu và nhãn
X = df.iloc[:, :-1]  # Tất cả các cột trừ cột cuối cùng
y = df.iloc[:, -1]   # Cột cuối cùng là nhãn

# Chia dữ liệu thành tập huấn luyện và tập kiểm thử
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình SVM
model = SVC(kernel='linear', C=1.0, probability=True)
model.fit(X_train, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Độ chính xác của mô hình: {accuracy:.2f}")

# Lưu mô hình đã huấn luyện
with open("gesture_model.pkl", "wb") as f:
    pickle.dump(model, f)
print("Mô hình đã được huấn luyện và lưu vào gesture_model.pkl")

Độ chính xác của mô hình: 1.00
Mô hình đã được huấn luyện và lưu vào gesture_model.pkl


In [10]:
# Load trained gesture model
with open("gesture_model.pkl", "rb") as f:
    model = pickle.load(f)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Initialize webcam
cap = cv2.VideoCapture(0)

print("Bắt đầu phân loại cử chỉ tay trong thời gian thực...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    gesture_text = "None"

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get and normalize hand landmark coordinates
            landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten().reshape(1, -1)

            # Predict gesture
            prediction = model.predict(landmarks)
            gesture_text = prediction[0]

            # Draw hand landmarks on frame
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=hand_landmarks,
                connections=mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec
            )

    cv2.putText(frame, f"Gesture: {gesture_text}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Real-time Gesture Classification", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Bắt đầu phân loại cử chỉ tay trong thời gian thực...


c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\PC\Deskop\Hand_Tracking_DarkFromAbyss\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but S